# **Data Collection**

## Objectives

* Fectch the date from Kaggle

## Inputs

* Kaggle JSON file - Authentication token

## Outputs

* Generate Dataset: input/datasets/cars_inventory

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

Select current directory

In [38]:
import os

# Change to the desired directory
os.chdir(r"C:\Users\Luis\Desktop\Code practice and projects\cars-customer-service")

# Verify the current working directory
current_dir = os.getcwd()
print(current_dir)

C:\Users\Luis\Desktop\Code practice and projects\cars-customer-service


Change current directory to the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

Confirm the new current directory

# Data Collection

Change the Kaggle configuration directory to the current working directory and set permissions for the Kaggle authentication JSON

In [39]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

'chmod' is not recognized as an internal or external command,
operable program or batch file.


Set the Kaggle dateset and download it.

In [45]:
KaggleDatasetPath = "anandmahajan84/used-cars-dealer-inventory-uk"
DestinationFolder = "inputsdatasets/cars_inventory"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}


Dataset URL: https://www.kaggle.com/datasets/anandmahajan84/used-cars-dealer-inventory-uk
License(s): GPL-2.0




  0%|          | 0.00/1.07M [00:00<?, ?B/s]
 94%|█████████▎| 1.00M/1.07M [00:00<00:00, 2.89MB/s]
100%|██████████| 1.07M/1.07M [00:00<00:00, 3.02MB/s]


---

# Data preparation

Data Cleaning

In [49]:
import os

def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')

    # Ensure the directory exists
    if not os.path.exists(my_data_dir):
        print(f"Error: Directory {my_data_dir} does not exist.")
        return
    
    # List all folders inside the given directory
    folders = os.listdir(my_data_dir)
    
    for folder in folders:
        folder_path = os.path.join(my_data_dir, folder)

        # Skip if not a directory (sometimes there can be files in the main directory)
        if not os.path.isdir(folder_path):
            print(f"Skipping {folder_path} as it is not a directory.")
            continue
        
        # List all files and subdirectories in the folder
        files = os.listdir(folder_path)
        image_count = 0
        non_image_count = 0
        
        for given_file in files:
            file_path = os.path.join(folder_path, given_file)
            
            # Skip if the given file is a directory (this avoids the 'Is a directory' error)
            if os.path.isdir(file_path):
                print(f"Skipping directory: {file_path}")
                continue

            # Check if the file ends with valid image extensions
            if not given_file.lower().endswith(image_extension):
                try:
                    # Attempt to remove the non-image file
                    os.remove(file_path)
                    non_image_count += 1
                    print(f"Removed non-image file: {file_path}")
                except Exception as e:
                    print(f"Error removing file {file_path}: {e}")
            else:
                image_count += 1
        
        # Print summary for the folder
        print(f"Folder: {folder} - contains {image_count} image files and {non_image_count} non-image files.")



---

## Split train validation test set

In [ ]:
import os
import shutil
import random

def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):
    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # Ensure the directory exists
    if not os.path.exists(my_data_dir):
        print(f"Error: Directory {my_data_dir} does not exist.")
        return

    # Get class labels
    labels = os.listdir(my_data_dir)  # It should get only the folder names
    if 'test' in labels:
        pass
    else:
        # Create train, validation, test folders with class label sub-folders
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(os.path.join(my_data_dir, folder, label), exist_ok=True)

        for label in labels:
            files = os.listdir(os.path.join(my_data_dir, label))
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                src_path = os.path.join(my_data_dir, label, file_name)
                if count <= train_set_files_qty:
                    # Move a given file to the train set
                    dst_path = os.path.join(my_data_dir, 'train', label, file_name)
                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # Move a given file to the validation set
                    dst_path = os.path.join(my_data_dir, 'validation', label, file_name)
                else:
                    # Move a given file to the test set
                    dst_path = os.path.join(my_data_dir, 'test', label, file_name)

                shutil.move(src_path, dst_path)
                count += 1

            # Remove the original label directory if it's empty
            os.rmdir(os.path.join(my_data_dir, label))

# Example usage

Devide the dataset in:
* Training set in 0.70 ratio
* Validation set in 0.10 ratio
* Test set in 0.20 ratio


In [64]:
split_train_validation_test_images(my_data_dir=f"inputsdatasets/cars_inventory",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )

---